<div style="text-align:center;font-size:22pt; font-weight:bold;color:white;border:solid black 1.5pt;background-color:#1e7263;">
    TensorBoard Callback Overview
</div>

In [1]:
# ======================================================================= #
# Course: Deep Learning Complete Course (CS-501)
# Author: Dr. Saad Laouadi
# Institution: Quant Coding Versity Academy
# Date: December 25, 2024
#
# ==========================================================
# Lesson: Learning Rate Adaptation with ReduceLROnPlateau
#         Synthetic Data Example
# ==========================================================
# ## Learning Objectives
# This example will enable you to:
# 1. Create synthetic data for learning rate adaptation
# 2. Implement ReduceLROnPlateau callback
# 3. Monitor learning rate changes during training
# 4. Visualize the impact of learning rate reduction
# 5. Compare training with and without adaptive learning rates
# =======================================================================
#          Copyright © Dr. Saad Laouadi 2024
# =======================================================================

In [2]:
# ==================================================== #
#        Load Required Libraries
# ==================================================== #

import os  
import shutil
from datetime import datetime
import io


# Disable Metal API Validation
os.environ["METAL_DEVICE_WRAPPER_TYPE"] = "0"  

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# import tensorflow
import tensorflow as tf

from tensorflow.keras.callbacks import ReduceLROnPlateau

# Set styling for better visualization
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("="*72)

%reload_ext watermark
%watermark -a "Dr. Saad Laouadi" -u -d -m

print("="*72)
print("Imported Packages and Their Versions:")
print("="*72)

%watermark -iv
print("="*72)

# Global Config
RANDOM_STATE = 101

Author: Dr. Saad Laouadi

Last updated: 2025-01-09

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 24.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Imported Packages and Their Versions:
numpy     : 1.26.4
seaborn   : 0.13.2
sklearn   : 1.5.1
matplotlib: 3.9.2
tensorflow: 2.16.2
pandas    : 2.2.2
keras     : 3.6.0



In [3]:
# ============================================================================ #
#                         Environment Path Configuration                       #
# ============================================================================ #
#
# Purpose:
#   Configure the system PATH to use Python executables from the active virtual 
#   environment instead of global installations.
#
# Usage:
#   1. First verify if configuration is needed by running: !which python
#   2. If the output shows the global Python installation rather than your 
#      virtual environment, execute this configuration block
#
# Note:
#   This configuration is typically only needed for JupyterLab Desktop or 
#   similar standalone installations. Web-based JupyterLab or properly 
#   configured environments should not require this adjustment.
# ============================================================================ #

import os
import sys

env_path = os.path.dirname(sys.executable)
os.environ['PATH'] = f"{env_path}:{os.environ['PATH']}"

In [4]:
# tensorboard cleanup   
import psutil
def cleanup_tensorboard():
    """
    Cleanup TensorBoard processes professionally
    """
    # Find and terminate TensorBoard processes
    for proc in psutil.process_iter(['pid', 'name']):
        try:
            # Look for tensorboard processes
            if 'tensorboard' in proc.info['name'].lower():
                # Terminate gracefully
                process = psutil.Process(proc.info['pid'])
                process.terminate()
                print(f"TensorBoard process {proc.info['pid']} terminated gracefully")
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            pass

### Notebook Roadmap

1. Create data
2. Train simple deep learning model with `tensorboard` callback
3. Use tensorboard tool to visualize training process
4. Stop tensorboard

In [5]:
# ================================================================= #
#          Test TensorBoard with Synthetic Data
# ================================================================= #

def generate_synthetic_data(n_samples=1000, random_state=42):
    """
    Generate synthetic regression data with two features.
    
    Parameters:
        n_samples (int): Number of samples to generate
        random_state (int): Random seed for reproducibility
        
    Returns:
        tuple: (X_train, X_test, y_train, y_test)
    """
    # Generate synthetic data
    X, y = make_regression(
        n_samples=n_samples,
        n_features=2,
        n_informative=2,
        noise=0.1,
        random_state=random_state
    )
    
    # Reshape y to be (n_samples, 1)
    y = y.reshape(-1, 1)
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=random_state
    )
    
    # Scale the features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, X_test, y_train, y_test

def create_simple_model():
    """
    Create a simple neural network for regression.
    
    Returns:
        tf.keras.Model: Compiled model
    """
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(2,)),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    
    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae', tf.keras.metrics.RootMeanSquaredError(name='rmse')]
    )
    
    return model

In [6]:
# Generate synthetic data
X_train, X_test, y_train, y_test = generate_synthetic_data()

# Create the model
model = create_simple_model()

# Set up TensorBoard
model_name="test_tensorboard"

# Create directory for logs
log_dir = os.path.join("logs", "fit", model_name, datetime.now().strftime('%Y%m%d-%H%M%S'))
os.makedirs(log_dir, exist_ok=True)

# Create TensorBoard callback
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=True,
    write_images=False,
    update_freq='epoch',
    profile_batch=(5, 10)
)

In [7]:
# Load TensorBoard extension
%load_ext tensorboard

In [8]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[tensorboard_callback],
    verbose=1
)

# Launch TensorBoard
%tensorboard --logdir logs/fit

Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 1433.7357 - mae: 29.8445 - rmse: 37.8161 - val_loss: 1621.5215 - val_mae: 31.6053 - val_rmse: 40.2681
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 1487.3281 - mae: 30.5533 - rmse: 38.5363 - val_loss: 1613.1566 - val_mae: 31.5205 - val_rmse: 40.1641
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1600.1937 - mae: 30.7971 - rmse: 39.9787 - val_loss: 1604.5652 - val_mae: 31.4333 - val_rmse: 40.0570
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1546.8121 - mae: 31.1174 - rmse: 39.3012 - val_loss: 1595.1746 - val_mae: 31.3328 - val_rmse: 39.9396
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1495.8247 - mae: 30.4601 - rmse: 38.6527 - val_loss: 1582.7705 - val_mae: 31.1979 - val_rmse: 39.7840
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1536.9329 - mae: 31.0749 - rmse: 39.1974 - val_loss: 1566.3914 - val_mae: 31.0196 - val_rmse: 39.5777
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━

In [9]:
# Evaluate the model on test set
test_results = model.evaluate(X_test, y_test, verbose=0)
print("\nTest Results:")
for metric_name, value in zip(model.metrics_names, test_results):
    print(f"{metric_name}: {value:.4f}")


Test Results:
loss: 28.8859
compile_metrics: 4.1993


In [10]:
# Use before starting a new TensorBoard session
cleanup_tensorboard()

In [13]:
# Start fresh TensorBoard
%reload_ext tensorboard

# Clear any previous logs (optional)
!rm -rf logs/fit

# !rm -rf logs (to remove the entire logs directory)

In [14]:
# Use tensorboar
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 99236), started 0:01:15 ago. (Use '!kill 99236' to kill it.)